## **Script di raccolta streaming con hashtag legati al Concerto del Primo Maggio - KAFKA Producer**


In [1]:
import tweepy
from kafka import KafkaProducer
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import Stream
import json
import time

from datetime import datetime, timedelta
def datetime_italiano(time):
    mytime = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    mytime += timedelta(hours=1)   # the tweets are timestamped in GMT timezone, while I am in +2 timezone
    return (mytime.strftime("%H:%M:%S"))

class MyListener(StreamListener):
    
  def __init__(self, producer, producer_topic):
    super().__init__()
    self.producer = producer
    self.producer_topic = producer_topic

  def on_status(self, status):
    if hasattr(status, "retweeted_status"):  # Controlla se è un retweet
        try: #prova a mantenere testo intero
            text=status.retweeted_status.extended_tweet["full_text"] 
        except AttributeError:
            text=status.retweeted_status.text
        retweet=True
    else: #non è un retweet
        try: #prova a mantenere testo intero
            text=status.extended_tweet["full_text"]
        except AttributeError:
            text=status.text
        retweet=False

    if status.place is not None : 
        try:
            name=status.place.name
        except Exception:
            name=False
        try:
            country_code=status.place.country_code
        except Exception:
            country_code=False
        try:
            centroid=str(status.place.centroid)
        except Exception:
            centroid=False
        try:
            bounding_box=str(status.place.bounding_box)
        except Exception:
            bounding_box=False
    
    else:
        name=False
        country_code=False
        centroid=False
        bounding_box=False
        
    #campi tweet da filtrare 
    tweet = {
        'user':{
            'user_id': status.user.id_str,
          'username': status.user.name,
            'user_location':status.user.location,
          'screen_name': status.user.screen_name,
            'user_followers':status.user.followers_count,
            #'utc_offset':status.user.utc_offset,
            'verified':status.user.verified,
            'user_creation':str(datetime.strptime(str(status.user.created_at),"%Y-%m-%d %H:%M:%S")), 
            'tweets_count':status.user.statuses_count, #sapendo quando è creato l'account si può creare un rateo
            'lang':status.user.lang
        },
        'tweet':{
            'tweet_id': status.id_str,
            'language':status.lang,
            'timestamp':datetime_italiano(str(status.created_at)), #normalize_timestamp(---)
            'text':text, 
            'retweet':retweet,
            'retweeted':status.retweeted,
            'retweet_count': status.retweet_count,
            'likes':status.favorite_count,
            'hashtags':status.entities["hashtags"], 
            'user_mentions':status.entities['user_mentions'], #altri utenti menzionati nel tweet
            'coordinates':status.coordinates,
            'place':{'id':name,
                    'country_code':country_code,
                    'centroide':centroid,
                    'bounding_box':bounding_box}
        }
    
        
    }
    
    if status.entities.get('hashtags') is not None:
      hashtags = status.entities.get('hashtags')
      tweet['hashtags'] = [ h.get('text') for h in hashtags ]
        
    self.producer.send(topic=self.producer_topic, value=tweet)
    
consumer_key = "3fR9kqAuUSv2pUvQkxhwbEbLA"
consumer_secret = "aHdhtchiFibJgq7xZErIuPOsw1NcwjmBQGSeDh4Wtnzlq0SktG"
access_token = "1054459819145510914-3IBdJxCioKzWyBWM3V3X9QxRmYv0FJ"
access_token_secret = "eeIAZUK04vxstezQCfblUmISoQ1nhj366QxbFZCWcSFqv"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
producer = KafkaProducer(
  bootstrap_servers=["localhost:9092"],
  value_serializer=lambda v: json.dumps(v).encode("utf-8"))

listener = MyListener(producer=producer, producer_topic="progettoPSY")
stream = Stream(auth = api.auth, listener=listener)

def go_streaming():
    import winsound
    i=0
    while i<30:
        try:
            stream.filter(track=["@primomaggioroma","#1M2021","#ConcertodelPrimoMaggio","#auditoriumparcodellamusica","#concertone","#concerto1maggio"]) 
        except Exception as e:
            print(str(i+1)+'_Error: '+str(e))
            duration = 2000  # milliseconds
            freq = 440  # Hz
            winsound.Beep(freq, duration)
            i+=1

In [2]:
go_streaming()

KeyboardInterrupt: 

In [ ]:
#NB lo script è in stato di errore poichè la procedura di streaming è stata stoppata manualmente a fine raccolta